In [1]:
from utils.decode_best_heuristics import get_best_heuristic
%load_ext autoreload
%autoreload 2

In [18]:
import json

with open('ejecuciones/CrossoverExplotacion_Reflection_Gemini2/heuristics_data.json', 'r') as file:
    heuristics_data = json.load(file)
with open('ejecuciones/CrossoverExplotacion_Reflection_Gemini2/final_information.json', 'r') as file:
    final_information = json.load(file)

In [ ]:
import pandas as pd

def get_best_global():

    file_path = "problems/jssp/greedy_solution.csv"
    data = pd.read_csv(file_path, sep=";")

    df_best_global = pd.DataFrame(columns=['instance', 'Separation', 'Balance', 'Makespan'])

    for index, row in data.iterrows():
        instance = row['instance']
        objective = row['objective']
        
        if instance not in df_best_global['instance'].values:
            new_row = pd.DataFrame({'instance': [instance], 'Separation': [None], 'Balance': [None], 'Makespan': [None]})
            df_best_global = pd.concat([df_best_global, new_row], ignore_index=True)
        
        df_best_global.loc[df_best_global['instance'] == instance, objective] = int(row[objective])
    return df_best_global

In [13]:
best = pd.read_csv("problems/jssp/greedy_solution.csv")

In [20]:
for id in final_information['best heuristics']:
    for heuristic in heuristics_data:
        if heuristic['id'] == id:
            for instance_name, evaluation in heuristic['evaluation'].items():
                for of_name, of_value in evaluation.items():
                    if of_name in ['Makespan', 'Balance']:
                        best_value = best.loc[best['instance'] == instance_name, of_name].iloc[0]
                        evaluation[of_name] = of_value / best_value
            print(heuristic)
            break

{'id': 47, 'code file': 'ejecuciones\\CrossoverExplotacion_Reflection_Gemini2/heuristics/heuristic47.py', 'valid': True, 'generation': 1, 'repair counter': 0, 'evaluation': {'mk00.txt': {'Makespan': 2.0, 'Separation': 0, 'Balance': 1.6666666666666667}, 'mk01.txt': {'Makespan': 2.55, 'Separation': 6, 'Balance': 1.9166666666666667}, 'mk02.txt': {'Makespan': 4.37037037037037, 'Separation': 33, 'Balance': 1.6153846153846154}, 'mk03.txt': {'Makespan': 4.200980392156863, 'Separation': 213, 'Balance': 1.3431372549019607}, 'mk04.txt': {'Makespan': 4.633333333333334, 'Separation': 86, 'Balance': 1.6}, 'mk05.txt': {'Makespan': 2.7752808988764044, 'Separation': 55, 'Balance': 1.2790697674418605}, 'mk06.txt': {'Makespan': 6.246376811594203, 'Separation': 53, 'Balance': 2.3541666666666665}, 'mk07.txt': {'Makespan': 2.9411764705882355, 'Separation': 56, 'Balance': 1.6906474820143884}, 'mk08.txt': {'Makespan': 4.167896678966789, 'Separation': 605, 'Balance': 1.1434034416826004}, 'mk09.txt': {'Makespa

In [9]:
best_global.to_csv('problems/jssp/greedy_solution.csv')